# Plan
1. First we need to take a look at the dataset and understand it(shape, size, features, types of features, null values)
2. Then we could do our EDA(exploratory data analysis) as boxplotting each feature to check for outliers, !! i just knew that the above is also considered EDA
